In [1]:
import torch
from modules.MyTrainer import NILMTrainer
from modules.helpers import create_tree_dir, save_report, train_model

from torch.utils.data import Dataset, DataLoader

from modules.MyDataSet import MyChunk, MyChunkList

with torch.no_grad():
    torch.cuda.empty_cache() 

Global seed set to 42


Device: cuda:0


# Project Settings

In [2]:
clean = True
ROOT = 'output'
dev_list = ['fridge', 'kettle', ]
mod_list = ['SimpleGru','SAED', 'WGRU', 'S2P', ]
tree_levels = {'root': 'TEST_TREE', 'l1': ['results'], 'l2': dev_list, 'l3': mod_list}
create_tree_dir(tree_levels=tree_levels, clean=clean)

root
all clean
/mnt/B40864F10864B450/WorkSpace/PHD/PHD_exps/NewPaper/torch_nilm/TEST_TREE
1


# Experiment settings

In [3]:
SAMPLE_PERIOD = 6
WINDOW = 50
device = 'fridge'
BATCH = 1024
experiment_name = 'Cat1'
EPOCHS = 1
ITERATIONS = 3

model_hparams={
              'SimpleGru':{'hidden_dim':16, 'dropout':0},
              'SAED':{},
               'WGRU':{'dropout':0},
               'S2P': {'window_size':WINDOW, 'dropout':0},
               }

# TrainLoader

In [4]:
d_name = 'UKDALE'
path = '/mnt/B40864F10864B450/WorkSpace/PHD/PHD_exps/data/{}/{}.h5'.format(d_name,d_name)
dates = ["2013-07-15","2013-07-16"]
building = 1
train_dataset = MyChunk(path=path, building=building, window_size=WINDOW,
                  device=device, dates=dates, sample_period=SAMPLE_PERIOD)
train_loader = DataLoader(train_dataset, batch_size=BATCH, 
                          shuffle=False, num_workers=8)
mmax = train_dataset.mmax

Implementation of torch dataset using NILMTK


<ipython-input-4-1ff83272af70>:5: FutureWarning: Converting timezone-aware DatetimeArray to timezone-naive ndarray with 'datetime64[ns]' dtype. In the future, this will return an ndarray with 'object' dtype where each element is a 'pandas.Timestamp' with the correct 'tz'.
	To accept the future behavior, pass 'dtype=object'.
	To keep the old behavior, pass 'dtype="datetime64[ns]"'.
  train_dataset = MyChunk(path=path, building=building, window_size=WINDOW,


# TestLoader

In [5]:
d_name = 'UKDALE'
path = '/mnt/B40864F10864B450/WorkSpace/PHD/PHD_exps/data/{}/{}.h5'.format(d_name,d_name)
dates = ["2013-08-01", "2013-08-02"]
building = 1
test_dataset = MyChunk(path=path, building=building, window_size=WINDOW,
                  device=device, dates=dates, test=True, mmax=train_dataset.mmax, sample_period=SAMPLE_PERIOD)
test_loader = DataLoader(test_dataset, batch_size=BATCH, 
                          shuffle=False, num_workers=8)
ground = test_dataset.meterchunk

Implementation of torch dataset using NILMTK


<ipython-input-5-eb27622b9bff>:5: FutureWarning: Converting timezone-aware DatetimeArray to timezone-naive ndarray with 'datetime64[ns]' dtype. In the future, this will return an ndarray with 'object' dtype where each element is a 'pandas.Timestamp' with the correct 'tz'.
	To accept the future behavior, pass 'dtype=object'.
	To keep the old behavior, pass 'dtype="datetime64[ns]"'.
  test_dataset = MyChunk(path=path, building=building, window_size=WINDOW,


# Run Experiment

In [6]:
eval_params={'device':device,
             'mmax':mmax,
             'groundtruth':ground}
for model_name in mod_list:
    print('#'*40)
    print('MODEL: ', model_name)
    print('#'*40)
    for iteration in range(1,ITERATIONS+1):
        print('#'*20)
        print('Iteration: ', iteration)
        print('#'*20)
        m_hparams, results, preds = train_model(model_name=model_name,
                                                model_hparams=model_hparams[model_name],
                                                train_loader=train_loader, 
                                                test_loader=test_loader,
                                                epochs=EPOCHS,
                                                eval_params=eval_params)
        save_report(ROOT, model_name, device, experiment_name,
                    iteration, results, preds, ground)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


########################################
MODEL:  SimpleGru
########################################
####################
Iteration:  1
####################



  | Name  | Type      | Params
------------------------------------
0 | model | SimpleGru | 39.9 K
------------------------------------
39.9 K    Trainable params
0         Non-trainable params
39.9 K    Total params
0.160     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

/home/nick/anaconda3/envs/pytorch_nilm/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: The {log:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.
# log on a step or aggregate epoch metric to the logger and/or progress bar (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...


  | Name  | Type | Params
-------------------------------
0 | model | S2P  | 2.6 M 
-------------------------------
2.6 M     Trainable params
0         Non-trainable params
2.6 M     Total params
10.437    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

#### model name: S2P ####
metrics: {'recall': 0.0, 'precision': nan, 'f1': nan, 'accuracy': 0.487, 'MAE': 47.778, 'RETE': 0.601}
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'metrics': {'MAE': 47.778,
             'RETE': 0.601,
             'accuracy': 0.487,
             'f1': nan,
             'precision': nan,
             'recall': 0.0},
 'model': 'S2P',
 'preds': array([0.00519349, 0.00519371, 0.00519367, ..., 0.00439823, 0.00439825,
       0.00439824]),
 'test_test_avg_loss': 0.00017896386270876974}
--------------------------------------------------------------------------------
